In [36]:
import MDAnalysis as md
from MDAnalysis.analysis import align
from MDAnalysis.analysis.density import density_from_Universe
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)
plt.rcParams["figure.figsize"] = cm2inch(16,16)
plt.rcParams.update({'font.size':8})
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [24]:
def calculate_density(universe, selection, delta=1.0, gridcenter=None, xdim=None, ydim=None, zdim=None):
    """The density is calculated for a selection. The MDAnalysis density function returns a 3D tensor,
    which is subsequently flatterned. Only the 2D grid is returned"""
    site_density = density_from_Universe(universe, delta=delta,
                atomselection=selection, update_selection=True, gridcenter=gridcenter,xdim=xdim, ydim=ydim, zdim=zdim)
    arr=site_density.grid
    flatten=np.zeros([np.shape(arr)[0],np.shape(arr)[1]])
    for k, i in enumerate(range(np.shape(arr)[0])):
        for v,j in enumerate(range(np.shape(arr)[1])):
                   flatten[k,v]=np.sum(arr[i,j,:])
    return flatten

In [25]:
def select_prio(resname,u,priority=["PO4", "GL1", "AM1"]):
    """Given a resname a priority selection is made only containing a single bead for the same lipid type"""
    selection=u.select_atoms("resname %s and name PO4 GM1 AM1 ROH GL1" %resname)
    if len(selection)!=selection.n_residues:
        beads=np.unique(selection.names)
        print("%s contains multiple of the listed bead types:" %resname)
        print("The bead with the highest priority is selected")
        bead_type=priority[np.amin([priority.index(v) for v in beads])]
        print("Which is %s" %bead_type)
    else:
        beads=np.unique(selection.names)
        bead_type=beads[0]
    new_selection=u.select_atoms("resname %s and name %s" %(resname, bead_type))
    return new_selection, bead_type


In [4]:
systems=["IA","PA","FA_G-prot","FA_no_G-prot"]
lipid_dir = {"FA_G-prot":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "FA_no_G-prot":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "IA":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6",
            "PA":"POPC, POPS, POPA, POPG, POPE, CHOL, DPGM3, DPSM, PAP6"}
unique_lipids = ['POPC','POPS', 'POPA', 'POPG', 'POPE', 'CHOL', 'DPGM3', 'DPSM', 'PAP6']
unique_lipid_labels = ['POPC', 'POPS', 'POPA', 'POPG', 'POPE', 'CHOL', 'GM3', 'SM', 'PIP2']

In [22]:
ref = md.Universe('GRO/IA_0_nowat_renumbered.gro')
TM123 = "resid 133-166 171-199 219-255 and name BB"

/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/au447022/anaconda3/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".for

In [31]:
for s in systems:
    u = md.Universe('GRO/{0:s}_0_nowat_renumbered.gro'.format(s), 'XTC/{0:s}_all_nowat_res_center_fit_last5.xtc'.format(s))
    alignment = align.AlignTraj(u, ref, filename='XTC/{0:s}_all_nowat_res_center_fit_last5_DENSMAP.xtc'.format(s), select=TM123)
    alignment.run()

In [37]:
density = {"FA_G-prot":"",
            "FA_no_G-prot":"",
            "IA":"",
            "PA":""}

In [38]:
for s in systems:
    u = md.Universe('GRO/{0:s}_0_nowat_renumbered.gro'.format(s), 'XTC/{0:s}_all_nowat_res_center_fit_last5_DENSMAP.xtc'.format(s))
    dens_loc = [] 
    for l in unique_lipids:
        sele =  select_prio(l,u)
        dens =  calculate_density(u, sele, delta=1.0)
        np.save('{0:s}_{1:s}_Density_mda.npy'.format(s,l), dens)
        dens_loc.append(dens)
    density[s] = dens_loc

POPC contains multiple of the listed bead types:
The bead with the highest priority is selected
Which is PO4


AttributeError: 'tuple' object has no attribute 'replace'